In [5]:
import requests
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token
from obi_notebook import get_projects

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

# Circuit ID

In [6]:
circuit_id = "13a54362-0c99-43fd-94e4-58bdb5b79466"

# Create SimulationForm using Cirucit

In [7]:


# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.CircuitSimulationScanConfig.empty_config()

# Info
info = obi.Info(campaign_name="O1 Simulation", campaign_description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(5)))
sim_conf.add(sim_neuron_set, name='L1All')

with_neuron_sets_and_timestamps = True
if with_neuron_sets_and_timestamps:
    all_neuron_set = obi.AllNeurons()
    sim_conf.add(all_neuron_set, name='AllNeurons')

    sync_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet2", elements=range(3)))
    sim_conf.add(sync_neuron_set, name='ID3')

    vpm_neuron_set = obi.nbS1VPMInputs(sample_percentage=100)
    sim_conf.add(vpm_neuron_set, name='nbS1VPMInputs')

    # Regular Timesteps
    regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
    sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
if with_neuron_sets_and_timestamps:
    poisson_input = obi.PoissonSpikeStimulus(duration=800.0, timestamps=regular_timestamps.ref, frequency=20, source_neuron_set=sim_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
else:
    poisson_input = obi.PoissonSpikeStimulus(duration=800.0, frequency=20)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# if with_neuron_sets_and_timestamps:
#     sync_input = obi.FullySynchronousSpikeStimulus(timestamps=regular_timestamps.ref, source_neuron_set=sync_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
# else:
#     sync_input = obi.FullySynchronousSpikeStimulus()
# sim_conf.add(sync_input, name='SynchronousInputStimulus')

if with_neuron_sets_and_timestamps:
    current_stim = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timestamps.ref, neuron_set=sim_neuron_set.ref, amplitude=0.1)
else:
    current_stim = obi.ConstantCurrentClampSomaticStimulus(amplitude=0.1)
sim_conf.add(current_stim, name='ConstantCurrentClampSomaticStimulus')


sinusoidal_stim = obi.SinusoidalPoissonSpikeStimulus(
      timestamp_offset=0,
      duration=3000.0,
      minimum_rate=0.00001,
      maximum_rate=10,
      modulation_frequency_hz=5,
      phase_degrees=0,
      random_seed=0
)
sim_conf.add(sinusoidal_stim, name='SinusoidalPoissonSpikeStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref)
sim_conf.add(voltage_recording, name='VoltageRecording')

if with_neuron_sets_and_timestamps:
    time_window_voltage_recording = obi.TimeWindowSomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=2000.0)
else:
    time_window_voltage_recording = obi.TimeWindowSomaVoltageRecording(start_time=0.0, end_time=2000.0)
sim_conf.add(time_window_voltage_recording, name='TimeWindowVoltageRecording')

# Initialization
if with_neuron_sets_and_timestamps:
    simulations_initialize = obi.CircuitSimulationScanConfig.Initialize(circuit=obi.CircuitFromID(id_str=circuit_id), 
                                                            node_set=sim_neuron_set.ref, 
                                                            simulation_length=sim_duration)
else:
    simulations_initialize = obi.CircuitSimulationScanConfig.Initialize(circuit=obi.CircuitFromID(id_str=circuit_id), 
                                                            simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

# Generate GridScanGenerationTask (locally) - Note strange issue described for relative paths

In [8]:
import tempfile

# Create a temporary directory (not used in this example, but can be useful for testing)
with tempfile.TemporaryDirectory() as temp_dir:

    """
    Very strange issue with adding assets when a relative path is used here. Need to look into after CNS.
    """
    # grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
    grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='/Users/james/Documents/obi/code/obi-output/circuit_simulations/grid_scan')
    # grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    grid_scan.execute(db_client=db_client)
    obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)


[2026-01-06 13:16:10,900] INFO: 
MULTIPLE VALUE PARAMETERS
[2026-01-06 13:16:10,901] INFO: No multiple value parameters found.
[2026-01-06 13:16:10,901] INFO: 
COORDINATE PARAMETERS
[2026-01-06 13:16:10,901] INFO: No coordinate parameters.
[2026-01-06 13:16:10,904] INFO: 1. Initializing simulation campaign in the database...
[2026-01-06 13:16:10,904] INFO: -- Register SimulationCampaign Entity
[2026-01-06 13:16:11,330] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation-campaign "HTTP/1.1 200 OK"
[2026-01-06 13:16:11,330] INFO: -- Upload campaign_generation_config
[2026-01-06 13:16:11,590] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation-campaign/a4e99f6a-04b7-4ffa-9b3f-cea2c729bb76/assets "HTTP/1.1 201 Created"
[2026-01-06 13:16:11,593] INFO: 2.0 Saving simulation 0 to database...
[2026-01-06 13:16:11,594] INFO: -- Register Simulation Entity
[2026-01-06 13:16:11,717] INFO: HTTP Request: POST https://staging

# Test endpoint. Service must be launched first (make run-local)

In [10]:
import requests

# Construct the full endpoint URL
# url = f"{obi_one_api_url}/generated/circuit-simulation-scan-config-generate-grid"
url = f"{obi_one_api_url}/declared/scan_config/grid-scan-coordinate-count"
# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request
# Placeholder example:
request_body = {
 'type': 'CircuitSimulationScanConfig',
 'timestamps': {'RegularTimestamps': {'type': 'RegularTimestamps',
   'start_time': 0.0,
   'number_of_repetitions': 3,
   'interval': 3000.0}},
 'stimuli': {
  'PoissonInputStimulus': {'type': 'PoissonSpikeStimulus',
   'timestamps': {'type': 'TimestampsReference',
    'block_dict_name': 'timestamps',
    'block_name': 'RegularTimestamps'},
   'source_neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'},
   'targeted_neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'},
   'duration': 800.0,
   'random_seed': 0,
   'frequency': 20.0}
   },
 'recordings': {'VoltageRecording': {'type': 'SomaVoltageRecording',
   'dt': 0.1,
   'neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'}}},
 'neuron_sets': {'L1All': {'type': 'IDNeuronSet',
    'sample_percentage': [30, 100],
   'sample_seed': 0,
   'neuron_ids': {'type': 'NamedTuple',
    'name': 'IDNeuronSet1',
    'elements': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)}}},
 'initialize': {'type': 'CircuitSimulationScanConfig.Initialize',
  'circuit': {'type': 'CircuitFromID',
   'id_str': circuit_id},
  'simulation_length': [2000.0],
  'node_set': {'type': 'NeuronSetReference',
   'block_dict_name': 'neuron_sets',
   'block_name': 'L1All'},
  'random_seed': 1,
  'extracellular_calcium_concentration': [1.1, 1.15],
  'v_init': -80.0},
 'info': {'type': 'Info',
  'campaign_name': 'No name provided',
  'campaign_description': 'No description provided'}
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    simulation_campaign_id = data
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

Success: 4


# Stage the Circuit

In [ ]:
from entitysdk.models import Circuit
circuit = db_client.get_entity(entity_id=circuit_id, entity_type=Circuit)

from entitysdk.staging.circuit import stage_circuit
from pathlib import Path
stage_circuit(client=db_client, model=circuit, output_dir=Path('temp_circ/'))

# Stage the Simulation

In [ ]:
from entitysdk.models import SimulationCampaign, Simulation
simulation_campaign = db_client.get_entity(entity_id=simulation_campaign_id, entity_type=SimulationCampaign)
simulation = db_client.get_entity(entity_id=simulation_campaign.simulations[0].id, entity_type=Simulation)

for asset in simulation.assets:
    if asset.content_type == "application/json":
        if asset.label == 'sonata_simulation_config':
            # Download the content into memory
            content = db_client.download_file(
                entity_id=simulation.id,
                entity_type=Simulation,
                asset_id=asset.id,
                output_path='simulation_config.json'
            )

from pathlib import Path
from entitysdk.staging.simulation import stage_simulation
stage_simulation(client=db_client, model=simulation, output_dir=Path('temp_sim'), circuit_config_path=Path("../temp_circ/circuit_config.json"))